In [33]:
import pandas as pd 
import datetime as dt 
import yfinance as yf 
import numpy as np 
from scipy.stats import norm
from curl_cffi import requests 
session = requests.Session(impersonate="chrome")

Setting up Variables 

In [37]:
contract_name = "TATAMOTORS250828C700"
underlying_symbol = "TATAMOTORS.NS"

#Set up dates 
current_date = dt.date.today()
expiry_date = dt.date(2025, 8, 28)
time_to_expiry = float((np.busday_count(current_date, expiry_date)+1))/252

#risk_free_rate 
risk_free_rate = 0.07

#Set up strike price
strike_price = 700

#Get Spot & Volatility
ticker = yf.Ticker(underlying_symbol,session=session)

Get Spot & Vol

In [22]:
hist_data = ticker.history(period="1y")
hist_data = hist_data["Close"]
hist_data.columns = ["Close"]

In [23]:
hist_data

Date
2024-08-05 00:00:00+05:30    1007.785217
2024-08-06 00:00:00+05:30    1005.108276
2024-08-07 00:00:00+05:30    1016.559814
2024-08-08 00:00:00+05:30    1032.869507
2024-08-09 00:00:00+05:30    1058.994873
                                ...     
2025-07-29 00:00:00+05:30     692.349976
2025-07-30 00:00:00+05:30     668.450012
2025-07-31 00:00:00+05:30     665.950012
2025-08-01 00:00:00+05:30     648.849976
2025-08-04 00:00:00+05:30     653.599976
Name: Close, Length: 250, dtype: float64

In [27]:
#Extracting Spot Price
spot_price = hist_data.iloc[-1]

In [30]:
#Calculating Annualized Volatility
daily_returns = hist_data.pct_change().dropna()
volatility = daily_returns.std() * np.sqrt(252)

In [34]:
# Black-Scholes Model Function
def black_scholes_mode(inputs):
    S = inputs['spot_price']
    K = inputs['strike_price']
    T = inputs['time_to_expiry']
    r = inputs['risk_free_rate']
    sigma = inputs['volatility']
    
    price = 0

    # Calculate d1 and d2
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if inputs['option_type'] == 'call':
        price = (S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2))
    else:
        price = (K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1))

    return price

In [38]:
#Prepare inputs for the model
inputs = {
    'spot_price': spot_price,
    'strike_price': strike_price,
    'time_to_expiry': time_to_expiry,
    'risk_free_rate': risk_free_rate,
    'volatility': volatility,
    'option_type': 'call'  }


In [40]:
# Calculate Call Option Price
call_option_price = black_scholes_mode(inputs)
print(f"The Black-Scholes price for the call option is: {call_option_price:.2f}")

The Black-Scholes price for the call option is: 6.57


In [41]:
# Calculate Put Option Price
inputs['option_type'] = 'put'
put_option_price = black_scholes_mode(inputs)
print(f"The Black-Scholes price for the put option is: {put_option_price:.2f}")

The Black-Scholes price for the put option is: 49.48
